# Install packages

In [1]:
%%capture
%%bash
pip install -U pip dask==2.16 numpy fsspec>=0.3.3 tqdm pyarrow

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d4xlarge' # change this
results_bucket = f"s3://xdss-benchmarks/benchmarks" # change this

name = 'dask'
data_path = 'datasets/taxi_parquet/'
output_file = f'{name}_{instance_type}.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks=benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]
          
def add_nan(name):
    for i in range(2):
        benchmarks['duration'].append(np.nan)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]

        

!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [3]:
import dask.dataframe as dd
import numpy as np

# Load data
data = dd.read_parquet(data_path, engine='pyarrow')
print(f"size: {len(data.vendor_id)} with {len(data.columns)} columns and {data.npartitions} partitions")

size: 1173057928 with 18 columns and 1174 partitions


In [4]:
def read_file_parquet(df=None):
    return dd.read_parquet(data_path, engine='pyarrow')

benchmark(read_file_parquet, df=data, name='read_file')

read_file took: 0.7032351493835449


0.7032351493835449

In [9]:
def count(df=None):
    # there is a dask bug - len(df) takes 20X time longer
    return len(df.vendor_id) 

benchmark(count, df=data, name='count')

count took: 42.307793855667114


42.307793855667114

In [10]:
def mean(df):
    return df.fare_amount.mean().compute()

benchmark(mean, df=data, name='mean')

mean took: 15.2627694606781


15.2627694606781

In [11]:
def standard_deviation(df):
    return df.fare_amount.std().compute()

benchmark(standard_deviation, df=data, name='standard deviation')

standard deviation took: 20.085809469223022


20.085809469223022

In [12]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean().compute()

benchmark(mean_of_sum, df=data, name='sum columns mean')

sum columns mean took: 16.069254875183105


16.069254875183105

In [13]:
# lazy evaulation - instant
def sum_columns(df):
    return (df.fare_amount + df.trip_distance)

benchmark(sum_columns, df=data, name='sum columns')

sum columns took: 0.0010197162628173828


0.0010197162628173828

In [14]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean().compute()

benchmark(mean_of_product, df=data, name='product columns mean')

product columns mean took: 15.952919483184814


15.952919483184814

In [15]:
# lazy evaulation - instant
def product(df):
    return df.fare_amount * df.trip_distance

benchmark(product, df=data, name='product columns')

product columns took: 0.0010495185852050781


0.0010495185852050781

In [16]:
def lazy_mean(df):
    df['lazy'] = df.fare_amount * df.trip_distance
    return df['lazy'].mean().compute()
    
benchmark(lazy_mean, df=data, name='lazy evaluation')  

lazy evaluation took: 748.9647119045258


748.9647119045258

In [8]:
def value_counts(df):
    return df.passenger_count.value_counts().compute()

benchmark(value_counts, df=data, name='value counts')

value counts took: 21.929911136627197


21.929911136627197

In [10]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean().compute()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')

arithmetic operation mean took: 41.69830632209778


41.69830632209778

In [11]:
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean()

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation')

arithmetic operation took: 0.019322872161865234


0.019322872161865234

In [12]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              }).compute()

benchmark(groupby_statistics, df=data, name='groupby statistics')

groupby statistics took: 691.1317853927612


691.1317853927612

In [6]:
other = groupby_statistics(data)

In [7]:
def join_count(df, other):
    return len(dd.merge(df, other, left_index=True, right_index=True))

benchmark(join_count, data, name='join count', other=other)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


join count took: 666.6230595111847


666.6230595111847

In [ ]:
# crashes
def join_data(df, other):
    return dd.merge(df, other, left_index=True, right_index=True)

add_nan('join')
# benchmark(join_count, data, name='join', other=other)

## Filtered data

In [10]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 7 MB


In [11]:
expr_filter = (data.pickup_longitude > long_min)  & (data.pickup_longitude < long_max) & \
                  (data.pickup_latitude > lat_min)    & (data.pickup_latitude < lat_max) & \
                  (data.dropoff_longitude > long_min) & (data.dropoff_longitude < long_max) & \
                  (data.dropoff_latitude > lat_min)   & (data.dropoff_latitude < lat_max)

def filter_data(df):
    return df[expr_filter]

benchmark(filter_data, data, name='filter data')

filter data took: 0.0002048015594482422


0.0002048015594482422

In [12]:
filtered = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

cleaned 0 mb


In [ ]:
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
benchmarks['filtered sum columns'] =  np.nan
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
add_nan('filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
add_nan('filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = groupby_statistics(filtered)
add_nan('filtered join')
benchmark(join_count, filtered, name='filtered join count', other=other)
print(name)
get_results()